# PKMine: retrieval of PBPK parameters from literature

## Import credentials

In [1]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import nest_asyncio
nest_asyncio.apply()

import os
from dotenv import load_dotenv
load_dotenv()
# Set keys as environment variable by running the following 
# pip install python-dotenv
# Create .env file in the same folder as jupyter notebook containing the following:
# LLAMA_CLOUD_API_KEY=your_actual_key_here
# HUGGINGFACE_API_KEY=your_actual_key_here

# API access to llama-cloud. 
LLAMA_CLOUD_API_KEY = os.getenv("LLAMA_CLOUD_API_KEY")
# API access to HuggingFace
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

# Check if the keys are loaded
print("Llama Cloud API Key loaded:", "Yes" if LLAMA_CLOUD_API_KEY else "No")
print("HuggingFace API Key loaded:", "Yes" if HUGGINGFACE_API_KEY else "No")

Llama Cloud API Key loaded: Yes
HuggingFace API Key loaded: Yes


## Set up embedding model and LLM

In [2]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM, HuggingFaceInferenceAPI
from transformers import AutoTokenizer, AutoModel
from huggingface_hub import login
import transformers
login(token=HUGGINGFACE_API_KEY)

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
#model = AutoModel.from_pretrained("HuggingFaceH4/zephyr-7b-alpha")
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")
transformers.logging.set_verbosity_debug()
llm=HuggingFaceLLM(model_name="HuggingFaceH4/zephyr-7b-alpha", tokenizer = tokenizer, device_map="auto")

/home/mcorradi/.conda/envs/ragwoe/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/mcorradi/.cache/huggingface/token
Login successful


loading configuration file config.json from cache at /home/mcorradi/.cache/huggingface/hub/models--HuggingFaceH4--zephyr-7b-alpha/snapshots/2ce2d025864af849b3e5029e2ec9d568eeda892d/config.json
Model config MistralConfig {
  "_name_or_path": "HuggingFaceH4/zephyr-7b-alpha",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 2,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.40.1",
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file model.safetensors from cache at /home/mcorradi/.cache/huggingface/hub/models--HuggingFaceH4

## Parse documents

In [3]:
from llama_parse import LlamaParse

documents = LlamaParse(result_type="markdown").load_data("../pdf/Hultman et al. - 2016 - Use of HμREL Human Coculture System for Prediction.pdf")

Started parsing the file under job_id 297b6aa8-40b8-457d-9c34-95a97593b639


In [4]:
from llama_index.core.node_parser import MarkdownElementNodeParser

node_parser = MarkdownElementNodeParser(
    llm=llm, num_workers=8
)

In [5]:
nodes = node_parser.get_nodes_from_documents(documents)

7it [00:00, 32988.91it/s]
  0%|          | 0/7 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
__root__
  Expecting ',' delimiter: line 29 column 6 (char 611) (type=value_error.jsondecode; msg=Expecting ',' delimiter; doc={
  "summary": "List of metabolites and their mass shifts in a metabolomics study.",
  "table_title": "Metabolite Mass Shifts",
  "table_id": "id_19",
  "columns": [
    {
      "col_name": "Compound",
      "col_type": "string"
    },
    {
      "col_name": "Mass Trace (M + H)",
      "col_type": "number"
    },
    {
      "col_name": "Mass Shift",
      "col_type": "number"
    },
    {
      "col_name": "Transformation",
      "col_type": "string"
    },
    {
      "col_name": "t (min)",
      "col_type": "number"
    },
    {
      "col_name": "Hep Suspension 2 h",
      "col_type": "number"
    }; pos=611; lineno=29; colno=6)
Traceback (most recent call last):
  File "/home/mcorradi/.conda/envs/ragwoe/lib/python3.9/site-pack

In [6]:
base_nodes, objects = node_parser.get_nodes_and_objects(nodes)

## Index documents

In [10]:
# Recursive indexing allows to store both text and tables separately
from llama_index.core import VectorStoreIndex
recursive_index = VectorStoreIndex(nodes=base_nodes + objects, embed_model=embed_model)

loading configuration file config.json from cache at /tmp/llama_index/models--BAAI--bge-small-en/snapshots/2275a7bdee235e9b4f01fa73aa60d3311983cfea/config.json
Model config BertConfig {
  "_name_or_path": "BAAI/bge-small-en",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.40.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file model.safetensors from cache at /tmp/llama_index/models--BAAI--bge-small-en/snapshots/2275a7b

In [12]:
recursive_query_engine = recursive_index.as_query_engine(llm=llm,similarity_top_k=5)

## Query documents over both text and tables

In [13]:
query = "What is the clearance in vitro value for imipramine?"

response = recursive_query_engine.query(query)
print(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 